In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np


class DataFrameAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("DataFrame Analyzer")

        # Frame pour les widgets
        self.frame = tk.Frame(root)
        self.frame.pack(padx=10, pady=10, side=tk.LEFT)

        # Frame pour les opérations
        self.operations_frame = tk.Frame(root)
        self.operations_frame.pack(padx=10, pady=10, side=tk.RIGHT)

        # Bouton pour charger un fichier
        self.load_button = tk.Button(self.frame, text="Charger un fichier", command=self.load_file)
        self.load_button.grid(row=0, column=0, pady=(0, 10))

        # Liste des opérations disponibles
        self.operations = ["Statistiques", "Valeur la plus fréquente", "Valeurs distinctes", "Valeurs manquantes",
                            "Histogrammes", "box plot"]

        # Label, Checkbuttons, et Text pour chaque colonne
        self.column_buttons = []

        # DataFrame pour stocker les résultats
        self.results_df = pd.DataFrame()

        # Fenêtre pour les histogrammes
        self.histogram_window = None
        self.histogram_canvas_frame = None
        self.histogram_row = 0

    def create_image_window(self):
        if self.histogram_window:
            self.histogram_window.destroy()

        # Créer une seule fenêtre pour les histogrammes
        self.histogram_window = tk.Toplevel(self.root)
        self.histogram_window.title("Histogrammes")

        # Frame pour les canvases des histogrammes
        self.histogram_canvas_frame = tk.Frame(self.histogram_window)
        self.histogram_canvas_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        # Ajouter un bouton de fermeture
        close_button = tk.Button(self.histogram_window, text="Fermer", command=self.histogram_window.destroy)
        close_button.pack(side=tk.BOTTOM)

        # Réinitialiser la ligne de la grille
        self.histogram_row = 0

    def create_image_canvas(self, column, chart):
        # Vérifier si la colonne existe dans le DataFrame
        if column not in self.df.columns:
            tk.messagebox.showerror("Erreur", f"La colonne {column} n'existe pas dans le DataFrame.")
            return

        # Obtenir les données de la colonne
        data = self.df[column]

        # Créer un canevas pour l'histogramme
        figure, ax = plt.subplots(figsize=(4, 3))
        if chart == "histogram":
            ax.hist(data, bins='auto')
            ax.set_title(f'Histogramme de {column}')
        elif chart == "box plot":
            ax.boxplot(data)
            ax.set_title(f'Box plot de {column}')

        # Ajouter le canevas à la fenêtre des histogrammes
        canvas = FigureCanvasTkAgg(figure, master=self.histogram_canvas_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.LEFT, padx=5, pady=5)

        # Incrémenter le suivi de la ligne
        self.histogram_row += 1

        # Si nous avons atteint trois histogrammes, passer à une nouvelle ligne
        if self.histogram_row % 3 == 0:
            self.histogram_canvas_frame = tk.Frame(self.histogram_window)
            self.histogram_canvas_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    def load_file(self):
        # Ouvrir une boîte de dialogue pour sélectionner un fichier
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv"), ("Excel Files", "*.xlsx")])

        if file_path:
            # Charger le fichier dans un DataFrame
            if file_path.endswith('.csv'):
                self.df = pd.read_csv(file_path)
            elif file_path.endswith('.xlsx'):
                self.df = pd.read_excel(file_path)
            else:
                tk.messagebox.showerror("Erreur", "Format de fichier non pris en charge.")
                return

            # Créer des boutons pour chaque colonne
            self.create_column_buttons()

    def create_column_buttons(self):
        # Supprimer les anciens boutons s'ils existent
        for button in self.column_buttons:
            button.destroy()

        self.column_buttons = []

        # Créer des boutons pour chaque colonne du DataFrame
        for i, column in enumerate(self.df.columns):
            button = tk.Button(self.frame, text=f"{column}", command=lambda col=column: self.show_operations(col))
            button.grid(row=i + 1, column=0, sticky='w')
            self.column_buttons.append(button)
        
        # Créer des boutons pour chaque colonne du DataFrame
        for i, column in enumerate(self.df.columns):
            button = tk.Button(self.frame, text=f"{column}", command=lambda col=column: self.show_operations(col))
            button.grid(row=i + 1, column=0, sticky='w')

            # Associer la fonction change_color au clic du bouton
            button.bind("<Button-1>", lambda event, btn=button: self.change_color(event, btn))

            self.column_buttons.append(button)

    def change_color(self, event, button):
        # Réinitialiser la couleur de tous les boutons
        for btn in self.column_buttons:
            btn.configure(bg='SystemButtonFace')

        # Mettre en surbrillance le bouton cliqué en changeant la couleur
        button.configure(bg='lightblue')

    def show_operations(self, column):
        # Créer des widgets pour les opérations
        operation_widgets = []

        # Ajouter les checkbox en fonction du type de données de la colonne
        column_type = self.df[column].dtype
        for operation in self.operations:
            if operation == "box plot" and not (pd.api.types.is_numeric_dtype(column_type) and np.issubdtype(self.df[column].dtype, np.number)):
                continue  # Ignorer "box plot" pour les colonnes non numériques
            if operation == "Statistiques" and not pd.api.types.is_numeric_dtype(column_type):
                continue  # Ignorer "Statistiques" pour les colonnes non numériques
            checkbox_var = tk.IntVar()
            checkbox = tk.Checkbutton(self.operations_frame, text=operation, variable=checkbox_var)
            checkbox.grid(sticky='w')
            operation_widgets.append((operation, checkbox_var))

        # Bouton pour exécuter les opérations
        analyze_button = tk.Button(self.operations_frame, text="Analyser", command=lambda: self.analyze_operations(column, operation_widgets))
        analyze_button.grid(sticky='w')
        operation_widgets.append(("Analyze Button", analyze_button))

        # Ajouter les widgets à la frame des opérations
        for widget in operation_widgets:
            self.column_buttons.append(widget)

    def analyze_operations(self, column, operation_widgets):
        selected_operations = [operation[0] for operation in operation_widgets if operation[1].get() == 1]

        # Effectuer les opérations sélectionnées
        result_texts = []

        for operation in selected_operations:
            if operation == "Valeur la plus fréquente":
                result_texts.append(f"Valeur la plus fréquente: {self.df[column].mode().iloc[0]}")
            elif operation == "Valeurs distinctes":
                result_texts.append(f"Valeurs distinctes: {', '.join(map(str, self.df[column].unique()))}")
            elif operation == "Valeurs manquantes":
                result_texts.append(f"Missing : {self.df[column].isnull().mean() * 100} %")
            elif operation == "Statistiques":
                if pd.api.types.is_numeric_dtype(self.df[column]) and np.issubdtype(self.df[column].dtype, np.number):
                    stats = self.df[column].describe()
                    result_texts.append(f"Statistiques: {stats.drop(stats.index[4:7])}")
                else:
                    result_texts.append(f"La colonne {column} n'est pas numérique.")
            elif operation == "Histogrammes":
                if not self.histogram_window:
                    self.create_image_window()
                try:
                    # Créer un nouvel histogramme dans la fenêtre existante
                    self.create_image_canvas(column, chart="histogram")
                except Exception as e:
                    result_texts.append(f"Histogramme impossible pour ce type de données")
            elif operation == "box plot":
                if pd.api.types.is_numeric_dtype(self.df[column]) and np.issubdtype(self.df[column].dtype, np.number):
                    if not self.histogram_window:
                        self.create_image_window()

                    # Créer un nouvel histogramme dans la fenêtre existante
                    self.create_image_canvas(column, chart="box plot")
                else:
                    result_texts.append(f"La colonne {column} n'est pas numérique.")
            else:
                result_texts.append("Opération non prise en charge")

        # Mettre à jour le texte des résultats
        result_text = tk.Text(self.operations_frame, height=3, width=30)
        result_text.grid(sticky='w')
        result_text.insert(tk.END, "\n".join(result_texts))


# Créer la fenêtre principale
root = tk.Tk()
app = DataFrameAnalyzerApp(root)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/Roberto/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-3-0f9265da1468>", line 127, in <lambda>
    button.bind("<Button-1>", lambda event, btn=button: self.change_color(event, btn))
  File "<ipython-input-3-0f9265da1468>", line 134, in change_color
    btn.configure(bg='SystemButtonFace')
AttributeError: 'tuple' object has no attribute 'configure'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/Roberto/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-3-0f9265da1468>", line 127, in <lambda>
    button.bind("<Button-1>", lambda event, btn=button: self.change_color(event, btn))
  File "<ipython-input-3-0f9265da1468>", line 134, in change_color
    btn.configure(bg='SystemButtonFace')
AttributeError: 'tuple' object has no attribut